## data preprocessing

In [28]:
import pandas as pd
train_data = pd.read_parquet('data/processed/train_data.parquet')
test_data  = pd.read_parquet('data/processed/test_data.parquet')

In [29]:
train_data.head()

,IncidentId,IncidentGrade,evidence_count,DetectorId_nunique,AlertTitle_nunique,DeviceId_nunique,Sha256_nunique,IpAddress_nunique,Url_nunique,AccountSid_nunique,...,EntityType_Url,EntityType_User,EvidenceRole_Impacted,EvidenceRole_Related,SuspicionLevel_Incriminated,SuspicionLevel_Suspicious,LastVerdict_Malicious,LastVerdict_NoThreatsFound,LastVerdict_Other,LastVerdict_Suspicious
0,0,TruePositive,29997,6,6,1,1,874,1,148,...,0,9081,9084,20913,0.0,0.0,0.0,0.0,0.0,0.0
1,2,BenignPositive,20525,113,934,11,1881,3,3,7,...,4,320,5484,15041,0.0,20453.0,6645.0,4.0,0.0,13814.0
2,3,TruePositive,3,1,1,1,1,2,1,2,...,0,1,2,1,0.0,0.0,0.0,0.0,0.0,0.0
3,7,BenignPositive,12252,8,19,3,3,2474,3,1721,...,4,1737,3477,8775,0.0,10.0,1.0,10.0,0.0,13.0
4,8,TruePositive,6,2,2,1,1,1,1,3,...,0,2,4,2,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
from sklearn.preprocessing import LabelEncoder
lable_encoder = LabelEncoder()
train_data["label"] = lable_encoder.fit_transform(train_data["IncidentGrade"])
test_data["label"] = lable_encoder.transform(test_data["IncidentGrade"])

In [31]:
lable_encoder.classes_

array(['BenignPositive', 'FalsePositive', 'TruePositive'], dtype=object)

In [32]:
train_data['label'].value_counts()

label
0    218131
1    135158
2     95612
Name: count, dtype: int64

In [33]:
X = train_data.drop(['IncidentId', 'IncidentGrade', 'label'], axis=1)
y = train_data["label"]

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.1, stratify=y, random_state=67)

X_test = test_data.drop(['IncidentId', 'IncidentGrade', 'label'], axis=1)
y_test = test_data["label"]

In [34]:
print(f"training shape : {X_train.shape}")
print(f"validation shape : {X_validation.shape}")
print(f"testing shape : {X_test.shape}")

training shape : (404010, 154)
validation shape : (44891, 154)
testing shape : (236267, 154)


In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

In [ ]:
import numpy as np
class_counts  = np.bincount(y_train)
class_weights = len(y_train) / (len(class_counts) * class_counts)